In [24]:
import torch
from torch import nn
from torch.utils.data import DataLoader,TensorDataset
from torch import optim
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix

In [6]:
df=pd.read_csv(r"C:\Users\saipr\Downloads\creditcard.csv\creditcard.csv")
df.dropna(inplace=True)
x=df[df.columns[:-1]]
y=df[df.columns[-1]]
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [39]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
y_test.value_counts()
smote=SMOTE(random_state=42)
x_train,y_train=smote.fit_resample(x_train,y_train)

In [40]:
x_train=torch.tensor(np.array(x_train),dtype=torch.float32)
x_test=torch.tensor(np.array(x_test),dtype=torch.float32)
y_test=torch.tensor(np.array(y_test),dtype=torch.float32)
y_train=torch.tensor(np.array(y_train),dtype=torch.float32)
train_data=TensorDataset(x_train,y_train)
test_data=TensorDataset(x_test,y_test)
train_loader=DataLoader(train_data,batch_size=64,shuffle=True)
test_loader=DataLoader(test_data,batch_size=64,shuffle=True)



In [41]:
class NNLog(nn.Module):
    def __init__(self,input_size=30,output_size=1):
        super().__init__()
        self.network=nn.Sequential(
            nn.Linear(input_size,output_size),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.network(x)
        

In [42]:
model=NNLog()
criterion=nn.BCELoss()
optimiser=optim.Adam(model.parameters(),lr=0.01)


In [44]:
for epoch in range(1000):
    for input,output in train_loader:
        pred_output=model(input)
        loss=criterion(pred_output.squeeze(),output)
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
    print(f"loss at epoch {epoch+1}:{loss}")
    if loss <10e-20:
        print(epoch)
        break
    

loss at epoch 1:50.0
loss at epoch 2:59.25925827026367
loss at epoch 3:53.703704833984375
loss at epoch 4:50.0
loss at epoch 5:53.703704833984375
loss at epoch 6:46.296295166015625
loss at epoch 7:59.25925827026367
loss at epoch 8:57.407405853271484
loss at epoch 9:48.14814758300781
loss at epoch 10:51.85185241699219
loss at epoch 11:40.74074172973633
loss at epoch 12:51.85185241699219
loss at epoch 13:61.11111068725586
loss at epoch 14:51.85185241699219
loss at epoch 15:51.85185241699219
loss at epoch 16:46.296295166015625
loss at epoch 17:51.85185241699219
loss at epoch 18:48.14814758300781
loss at epoch 19:51.85185241699219
loss at epoch 20:55.55555725097656
loss at epoch 21:48.14814758300781
loss at epoch 22:48.14814758300781
loss at epoch 23:44.44444274902344
loss at epoch 24:42.592594146728516
loss at epoch 25:48.14814758300781
loss at epoch 26:57.407405853271484
loss at epoch 27:55.55555725097656
loss at epoch 28:37.03703689575195
loss at epoch 29:33.33333206176758
loss at epoch

KeyboardInterrupt: 

In [ ]:
def accuracy(model,loader):
    model.eval()
    y_true,y_pred=[],[]
    with torch.no_grad():
        for input,output in loader:
            pred_output=model(input)
            pred_output=pred_output.squeeze()
            pred_output_np=pred_output.detach().cpu().numpy().flatten()
            output_np=output.detach().cpu().numpy().flatten()
            y_true.extend(output_np.tolist())
            y_pred.extend(pred_output_np.tolist())
    model.train()
    y_pred=list(1 if k>=0.5 else 0 for k in y_pred )
    t1=sum(1 for j in y_true if j==1)
    t0=len(y_true)-t1
    c1=sum(1 for j,k in zip(y_true,y_pred) if j==1 and k==1)
    c0=sum(1 for j,k in zip(y_true,y_pred) if j==0 and k==0)
    print("precision_score")
    print(f"class1:{c1}/{t1},class0:{c0}/{t0}")
    print("classification_report:")
    print(classification_report(y_true,y_pred))
    print('confusion matrix:')
    print(confusion_matrix(y_true,y_pred))
    


In [ ]:
print(accuracy(model,train_loader))


precision_score
class1:0/394,class0:227451/227451
classification_report:


C:\Users\saipr\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\saipr\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\saipr\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    227451
         1.0       0.00      0.00      0.00       394

    accuracy                           1.00    227845
   macro avg       0.50      0.50      0.50    227845
weighted avg       1.00      1.00      1.00    227845

confusion matrix:
[[227451      0]
 [   394      0]]
None


In [ ]:
print(accuracy(model,test_loader))